# Exercise: Analyzing Chinook Database

Preparation I've done:
 - Retrieve the dataset and load it
 - Load the %sql extension and point it at the database
 - Display the tables and an example query

Additional steps you might take:
 - Add libraries for visualization (matplotlib, seaborn, plotly)
 - Add libraries for statistics (numpy)
 - Explore the dataset using SQL and/or pandas

----

1. Retrieve a list of all the tracks in the database, displaying only the track name and the name of the album it belongs to. Limit the result to the first 5 rows.
   > Operations: `SELECT`
2. Find the total number of customers from each country. Display the country name and the corresponding count. Order the results by the count in descending order.
   > Operations: `SELECT`, `COUNT`, `GROUP BY`, `ORDER BY`
3. Identify the top 5 genres with the highest number of tracks. Display the genre name along with the total number of tracks for each genre.
   > Operations: `SELECT`, `COUNT`, `GROUP BY`, `ORDER BY`
4. Determine the average invoice total for each customer, considering both the album and individual track purchases. Display the customer's first and last name along with the average invoice total. Order the results by the average invoice total in descending order.
   > Operations: `SELECT`, `AVG`, `JOIN`, `GROUP BY`, `ORDER BY`
5. Identify the customer who spent the most on music purchases. Display the customer's first and last name, along with the total amount spent.
   > Operations: `SELECT`, `SUM`, `JOIN`, `GROUP BY`, `ORDER BY`, `LIMIT`

In [1]:
# Import packages
import sqlalchemy
import matplotlib
import seaborn as sns
import plotly
import statistics


In [2]:
# Load chinook dataset and query it using SQL magic into pandas dataframes
import pandas as pd
import sqlite3
%load_ext sql

# Load data
conn = sqlite3.connect("chinook.sqlite")

# Tell %sql about the database
%sql sqlite:///chinook.sqlite

# List tables in database
query = "SELECT name FROM sqlite_master WHERE type='table';"

# Read data into a Pandas DataFrame
tables = %sql $query

# Print head
display(tables)

# Query to get the first 5 rows of the `albums` table
result = %sql SELECT * FROM albums LIMIT 5;

# Display query result, note that Pandas DataFrame is returned!
display(result)


 * sqlite:///chinook.sqlite
Done.


name
albums
sqlite_sequence
artists
customers
employees
genres
invoices
invoice_items
media_types
playlists


 * sqlite:///chinook.sqlite
Done.


AlbumId,Title,ArtistId
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3


### 1. Retrieve a list of all the tracks in the database, displaying only the track name and the name of the album it belongs to. Limit the result to the first 5 rows.
###  > Operations: `SELECT`

In [3]:
query = %sql SELECT * FROM tracks LIMIT 5;
display(query)



 * sqlite:///chinook.sqlite
Done.


TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman",230619,3990994,0.99
4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman",252051,4331779,0.99
5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


In [4]:
first_5_rows = %sql SELECT tracks.Name as track_name, AlbumId as album_name\
FROM tracks LIMIT 5;
display(first_5_rows)


 * sqlite:///chinook.sqlite
Done.


track_name,album_name
For Those About To Rock (We Salute You),1
Balls to the Wall,2
Fast As a Shark,3
Restless and Wild,3
Princess of the Dawn,3


### 2. Find the total number of customers from each country. Display the country name and the corresponding count. Order the results by the count in descending order.
### > Operations: `SELECT`, `COUNT`, `GROUP BY`, `ORDER BY`

In [5]:
nuber_of_customer = %sql SELECT DISTINCT Country, count(Customerid) as total_number\
FROM customers\
GROUP BY Country\
ORDER BY total_number DESC;

display(nuber_of_customer)

 * sqlite:///chinook.sqlite
Done.


Country,total_number
USA,13
Canada,8
France,5
Brazil,5
Germany,4
United Kingdom,3
Portugal,2
India,2
Czech Republic,2
Sweden,1


### 3. Identify the top 5 genres with the highest number of tracks. Display the genre name along with the total number of tracks for each genre.
###    > Operations: `SELECT`, `COUNT`, `GROUP BY`, `ORDER BY`

In [6]:
number_of_tracks = %sql SELECT DISTINCT genres.name, count(TrackId) as number_tracks\
FROM tracks LEFT JOIN genres ON tracks.GenreId = genres.GenreId\
GROUP BY genres.name\
ORDER BY number_tracks DESC LIMIT 5;

display(number_of_tracks)


 * sqlite:///chinook.sqlite
Done.


Name,number_tracks
Rock,1297
Latin,579
Metal,374
Alternative & Punk,332
Jazz,130


### 4. Determine the average invoice total for each customer, considering both the album and individual track purchases. Display the customer's first and last name along with the average invoice total. Order the results by the average invoice total in descending order.
###   > Operations: `SELECT`, `AVG`, `JOIN`, `GROUP BY`, `ORDER BY`

In [7]:
query = %sql SELECT * FROM invoices LIMIT 5;
display(query)

 * sqlite:///chinook.sqlite
Done.


InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86


In [8]:
avg_invoice = %sql SELECT customers.FirstName, customers.LastName, AVG(Total) as average_invoice\
FROM invoices LEFT JOIN customers ON invoices.CustomerId = customers.CustomerId\
GROUP BY invoices.CustomerId\
ORDER by average_invoice DESC;

display(avg_invoice)

 * sqlite:///chinook.sqlite
Done.


FirstName,LastName,average_invoice
Helena,Holý,7.088571428571429
Richard,Cunningham,6.802857142857143
Luis,Rojas,6.659999999999999
Ladislav,Kovács,6.517142857142857
Hugh,O'Reilly,6.517142857142857
Julia,Barnett,6.231428571428572
Frank,Ralston,6.231428571428571
Fynn,Zimmermann,6.231428571428571
Puja,Srivastava,6.1066666666666665
Astrid,Gruber,6.088571428571428


### 5. Identify the customer who spent the most on music purchases. Display the customer's first and last name, along with the total amount spent.
###   > Operations: `SELECT`, `SUM`, `JOIN`, `GROUP BY`, `ORDER BY`, `LIMIT`

In [9]:
primary_customer = %sql SELECT customers.FirstName, customers.LastName, SUM(invoices.Total) as total_amount\
FROM invoices LEFT JOIN customers ON invoices.CustomerId = customers.CustomerId\
GROUP BY invoices.CustomerID\
ORDER BY total_amount DESC LIMIT 1;

display(primary_customer)

 * sqlite:///chinook.sqlite
Done.


FirstName,LastName,total_amount
Helena,Holý,49.620000000000005
